# Phân loại spam email

Dữ liệu: từ khóa trong email, tần suất, độ dài tiêu đề.

SVM tìm “siêu phẳng” phân chia spam và không spam.

Giúp lọc thư rác trong Gmail, Outlook.

## Huấn luyện SVM (Linear SVM) với 3 đặc trưng số

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# 1) Đọc dữ liệu
df = pd.read_csv("spam_email_svm.csv")
X = df[["KeywordCount", "KeywordFreq", "SubjectLen"]]
y = df["IsSpam"]

# 2) Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# 3) Pipeline: Chuẩn hóa + Linear SVM
model = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(C=1.0, random_state=42)) # nhanh, hiệu quả cho dữ liệu thưa/đặc trưng số
])
model.fit(X_train, y_train)

# 4) Đánh giá
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9666666666666667
Confusion matrix:
 [[88  0]
 [ 5 57]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        88
           1       1.00      0.92      0.96        62

    accuracy                           0.97       150
   macro avg       0.97      0.96      0.97       150
weighted avg       0.97      0.97      0.97       150



## Dự đoán email mới

In [3]:
new_email = pd.DataFrame([{
    "KeywordCount": 5,
    "KeywordFreq": 9,     # trên 100 từ
    "SubjectLen": 32
}])

pred = model.predict(new_email)[0]
print("Spam?" , bool(pred))

Spam? True


# Tùy chọn nâng cao (nếu bạn muốn “giống thực tế” hơn)
## A. Lấy xác suất dự đoán (Platt scaling)

In [4]:
from sklearn.calibration import CalibratedClassifierCV

calibrated = CalibratedClassifierCV(model.named_steps["svm"], method="sigmoid", cv=5)
# fit lại với dữ liệu đã scale
X_train_scaled = model.named_steps["scaler"].fit_transform(X_train)
calibrated.fit(X_train_scaled, y_train)

# dự đoán xác suất
X_test_scaled = model.named_steps["scaler"].transform(X_test)
proba = calibrated.predict_proba(X_test_scaled)[:, 1]
print("Ví dụ 10 xác suất đầu:", proba[:10])

Ví dụ 10 xác suất đầu: [0.99979735 0.13368306 0.99085949 0.95482187 0.01813564 0.05063842
 0.52517136 0.11352224 0.9998621  0.05050412]


## B. Kết hợp TF-IDF từ Subject/Body + SVM (thực chiến hơn)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [6]:
# Vector hóa text + ghép numeric features
text_features = ["Subject", "Body"]
num_features = ["KeywordCount", "KeywordFreq", "SubjectLen"]

preprocess = ColumnTransformer([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3), "Body"), # có thể dùng cả "Subject"
    ("passthrough", "passthrough", num_features)
], remainder="drop")

clf = Pipeline([
    ("prep", preprocess),
    ("svm", LinearSVC(C=1.0, random_state=42))
])

X_all = df[text_features + num_features]
y_all = df["IsSpam"]

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.25, stratify=y_all, random_state=42
)
clf.fit(X_train, y_train)
print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 0.96


# Nhận diện cảm xúc trong phim 
Input: khuôn mặt diễn viên (ảnh). 
SVM phân loại: vui, buồn, giận dữ, sợ hãi. 
Dùng trong phân tích hành vi người xem hoặc robot giao tiếp.

## Train/Test SVM + đánh giá

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [8]:
# 1) Đọc dữ liệu
df = pd.read_csv("emotion_embeddings_svm.csv")
X = df.filter(like="f").values    # f000..f063
y = df["emotion"].values

# 2) Chia tập
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# 3) Pipeline: Chuẩn hóa + SVM RBF (có xác suất)
model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  # embeddings đã normalize; with_mean=False giữ ổn định
    ("svm", SVC(kernel="rbf", C=3.0, gamma="scale", probability=True, random_state=42))
])

# 4) Huấn luyện
model.fit(X_train, y_train)

# 5) Đánh giá
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 1.0
Confusion matrix:
 [[23  0  0  0]
 [ 0 22  0  0]
 [ 0  0 23  0]
 [ 0  0  0 22]]
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00        23
     fearful       1.00      1.00      1.00        22
       happy       1.00      1.00      1.00        23
         sad       1.00      1.00      1.00        22

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90



## Dự đoán cho mẫu mới (embedding 64-D)

In [9]:
import numpy as np

# Ví dụ embedding mới (thay bằng vector thật của bạn)
x_new = np.random.normal(0, 1, size=(1, X.shape[1]))
x_new = x_new / np.linalg.norm(x_new, axis=1, keepdims=True)

pred = model.predict(x_new)[0]
proba = model.predict_proba(x_new).max()
print("Emotion:", pred, "| Confidence:", round(proba, 3))

Emotion: sad | Confidence: 0.399


## (Tuỳ chọn) Tối ưu nhanh bằng GridSearch

In [10]:
from sklearn.model_selection import GridSearchCV

params = {
    "svm__C": [1.0, 3.0, 5.0],
    "svm__gamma": ["scale", 0.01, 0.05, 0.1],
    "svm__kernel": ["rbf"]
}
grid = GridSearchCV(model, params, cv=5, n_jobs=-1)
grid.fit(X, y)
print("Best params:", grid.best_params_, "| CV score:", grid.best_score_)

Best params: {'svm__C': 1.0, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'} | CV score: 1.0
